# Apuntes

In [ ]:
# Librerías:
import pandas as pd             # Manejo de datos
import numpy as np              # Manejo de arrays n-dimensionales y func. matemáticas
import scipy                    # Manejo de func. matemáticas y distrib. de probabilidad
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import random                   # Generación aleatoria
import matplotlib.pyplot as plt # Generación de gráficos

In [ ]:
# Base de datos de prueba
from sklearn.datasets import load_iris

In [ ]:
### Abrir la Base de Datos: ###

# bd = pd.read_csv('C:/Users/Diego/OneDrive - Universidad Rey Juan Carlos/Documentos/GIA_URJC/Curso 2023-24/G.-IA/G.-IA/Curso_2/Cuatri_2/AprendizajeAutomatico_1/Apuntes/california_housing_apuntes.csv', sep=',')

# Base de datos de prueba:
iris_data = load_iris()
bd = pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
bd['species'] = pd.Categorical.from_codes(iris_data.target, iris_data.target_names)

# Guardar tamaño de bd:
N, D = bd.shape

In [ ]:
### Dividir en Train-Test: ###

fraction_test = 0.2

# Crea una lista con los índices de la bd:
ind = bd.index.tolist()
# Desordena los índices:
random.shuffle(ind)

# Calcula la cantidad de ejemplos que se guardan en test:
N_test = int(N * fraction_test)

# Divide los datos:
test_df = bd.iloc[ind[:N_test]]
train_df = bd.iloc[ind[N_test:]]


In [ ]:
# Imprime el tamaño de las bd:
print(f'BD size: ({N}, {D})\nTrain size: {train_df.shape} \t Test size: {test_df.shape}')

In [ ]:
# Guardar los datos de train y Test en un directorio:

# flag_save = True
# if flag_save:
#     train_folder = 'C:/Users/Diego/OneDrive - Universidad Rey Juan Carlos/Documentos/GIA_URJC/Curso 2023-24/G.-IA/G.-IA/Curso_2/Cuatri_2/AprendizajeAutomatico_1/Apuntes/'
#     train_name   = 'train_df_apuntes.csv'
#     train_df.to_csv(train_folder + train_name, sep=';', header=True)

# if flag_save:
#     test_folder = 'C:/Users/Diego/OneDrive - Universidad Rey Juan Carlos/Documentos/GIA_URJC/Curso 2023-24/G.-IA/G.-IA/Curso_2/Cuatri_2/AprendizajeAutomatico_1/Apuntes/'
#     test_name   = 'test_apuntes.csv'
#     test_df.to_csv(test_folder + test_name, sep=';', header=True)

In [ ]:
# Información de los datos para ver si hay que codificar:
train_df.info()

### Tipos de datos: Atributos 'continuos', 'discretos' y 'categóricos' ###

In [ ]:
### La codificación de los tipos categóricos no siempre hay que hacerla por la falta de estos###
# 1) Averiguamos las columnas categóricas:
cat_cols = train_df.select_dtypes(include='category').columns.tolist()

# 2) Creamos un dataframe con las columnas categóricas:
train_cod = train_df

# 3) Codificamos la fila, a la vez que creamos una diccionario de diccionarios para descodificar en el futuro
dict_decode={}
for col in cat_cols:
  codes = train_df[col].cat.codes                   # Codifica la serie del df actual
  code_to_categ = dict(zip(codes,train_df[col]))    # Crea la relación de codificación
  train_cod[col] = codes                            # Asigna el valor codificado a la serie
  dict_decode[col] = code_to_categ                  # Añade la decodificación al diccionario

print(f'Columnas categóricas: {cat_cols}\n{train_cod.head()}\nDecodificación: {dict_decode}')

In [ ]:
### Tambien se puede codificar por One-Hot. Elegir cómo nos viene mejor. ###
### Codificación One-hot: crea una columna por cada categoría validando con 1 y 0 ###
OneHot_codification = train_df
for col in cat_cols:
    OneHot_codification = pd.concat([OneHot_codification, pd.get_dummies(OneHot_codification[col])], axis=1)

OneHot_codification

In [ ]:
# Ver la información del df
print(train_cod.info(), '\n')        # Indica el tamaño, los tipos de datos y cuántos son 'Na' de cada atributo
print(train_cod.describe(), '\n')    # Indica la descripción estadística básica excepto la moda de cada atributo en forma de tabla
train_cod.mode(axis=0, dropna=False) # Devuelve una tabla con las modas de cada atributo

In [ ]:
# Búsqueda de valores NaN:
missing_data = train_cod.isna()
missing_data.sum()

missing_values_per_column = missing_data.sum(axis=0)    # 'NA' por cada columna
missing_values_per_row = missing_data.sum(axis=1)       # 'NA' por cada fila

mask_mayorq0 = missing_values_per_column > 0            # Crea una máscara de Pandas para indicar si hay columnas con NA
mask_mayorq1 = missing_values_per_row > 0               # Crea una máscara de Pandas para indicar si hay filas con NA

print(f'Columnas con valores nulos:\n{missing_values_per_column[mask_mayorq0]}\n')
print(f'Filas con valores nulos:\n{missing_values_per_row[mask_mayorq1]}\n')

missing_count_row = missing_values_per_row.value_counts().sort_index()
print(f'Valores NaN en cada fila:\n{missing_count_row}')
missing_count_col = missing_values_per_column.value_counts().sort_index()
print(f'Valores NaN en cada columna:\n{missing_count_col}')

##### Se puede hacer una de las siguientes cosas:
- Eliminar filas/columnas Nan
- Imputación univariada
- Imputación multivariante

In [ ]:
### Eliminar filas y columnas con NaN excesivos ###

# Eliminar filas:
if missing_count_row.index[-1] > 0:                                         # Calcular la cantidad máxima de valores nulos por fila
    mask_toDrop = missing_values_per_row >= missing_count_row.index[-1]     # Filtro que busca las filas con el número de valores perdidos máximo
    drop_list_row = missing_values_per_row[mask_toDrop].index.tolist()      # Crea una lista de índices de las filas que cumplen con la condición
    train_cod.drop(drop_list_row, inplace=True)                             # Eliminar las filas guardadas en 'drop_list' del DataFrame original

# Eliminar columnas:                                      
if missing_count_col.index[-1] > 0:                                     # Calcular la cantidad máxima de valores nulos por columna
    mask_toDrop = missing_values_per_column >= missing_count_col.index[-1]  # Filtro que busca las filas con el número de valores perdidos máximo
    drop_list_col = missing_values_per_column[mask_toDrop].index.tolist()   # Crea una lista de índices de las columnas que cumplen con la condición
    train_cod.drop(drop_list_col, inplace=True)                             # Eliminar las columnas guardadas en 'drop_list' del DataFrame original, al usar implace no hace falta poner 'df = ...'

# Tamaño del DataFrame redimensionado:
train_cod.shape

In [ ]:
### Imputación univariada de los datos NaN: ###

train_cod.fillna(train_cod.median(axis=0), inplace=True)    # Sustituye los valores NaN por el valore de la mediana
train_cod.shape
# Se pueden sustituir por la media o el valor más repetido (moda)
# El problema de esta técnica es que puede crear ejemplos imposibles como decir que un hombre está embarazado

In [ ]:
### Imputación multivariante de los datos NaN: ###

imputer = IterativeImputer()
train_imputed = imputer.fit_transform(train_cod)

# Convertir de nuevo a DataFrame
train_imputed = pd.DataFrame(train_imputed, columns=train_cod.columns)

# Recuento de NaN:
train_imputed.isna().sum()

##### 

In [ ]:
### Ingeniería de características ###

# Crea una tabla con las correlaciones de las diferentes variables:
correlation_matrix = train_imputed.corr()

# Aumento de la dimensionalidad de las variables con baja correlación:
train_df_dim = train_imputed
degree = 2
interaction_only = True

polyf = PolynomialFeatures(degree=degree, interaction_only=interaction_only, include_bias=False)
polyf.set_output(transform="pandas")


# No se puede hacer esta línea de comando con el test NUNCA:
polyf.fit(train_cod)
# Adapta los datos a las nuevas características:
train_df_dim = polyf.transform(train_cod)

In [ ]:
print(f'Dataframe aumentado:\n{train_df_dim}')

In [ ]:
train_df_dim.corr()

In [ ]:
# # Mínimo de relación para separar entre una lista y otra:
# CORR_MIN = 0.6
# # Crear listas para guardar las características:
# high_corr, low_corr = [], []

# # Dividir las correlaciones en las listas:
# for column in correlation_matrix.columns:
#     for index in correlation_matrix.index:
#         if index == column:
#             continue
#         corr = correlation_matrix.loc[index, column]
#         if corr >= CORR_MIN or corr <= -CORR_MIN:
#             high_corr.append((index, column))
#         elif -CORR_MIN < corr < CORR_MIN:
#             low_corr.append((index, column))

# # Eliminar las correlaciones repetidas:
# high_corr = list(set(high_corr))
# low_corr = list(set(low_corr))